[PlayTypes](https://statsapi.web.nhl.com/api/v1/playTypes)
[Example game](https://statsapi.web.nhl.com/api/v1/game/2017020001/feed/live)
[Example team](https://statsapi.web.nhl.com/api/v1/teams/10)

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
if pathlib.Path().resolve().name == 'notebooks':
    %cd ..
%pwd

/Users/alexandreganito/Documents/Data science/Project/NHL


'/Users/alexandreganito/Documents/Data science/Project/NHL'

In [3]:
from datetime import datetime, time
from pydantic import BaseModel, model_validator, field_validator
from ipywidgets import widgets, interact, IntSlider
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, clear_output
from typing import Optional, Dict
import math
from enum import Enum
import pandas as pd
from src.data import load_data, load_processed_data, load_df_shots
from src.models import Side

# Raw data examples


In [5]:
raw = load_data(2016)

Season 2016 successfully loaded from file


In [36]:
for play in raw['regulars'][0]['liveData']['plays']['allPlays']:
  if play['result']['event'] == 'Shot':
    shot_exemple = play
  if play['result']['event'] == 'Goal':
    goal_exemple = play

shot_exemple

{'players': [{'player': {'id': 8476879,
    'fullName': 'Cody Ceci',
    'link': '/api/v1/people/8476879'},
   'playerType': 'Shooter'},
  {'player': {'id': 8475883,
    'fullName': 'Frederik Andersen',
    'link': '/api/v1/people/8475883'},
   'playerType': 'Goalie'}],
 'result': {'event': 'Shot',
  'eventCode': 'OTT867',
  'eventTypeId': 'SHOT',
  'description': 'Cody Ceci Slap Shot saved by Frederik Andersen',
  'secondaryType': 'Slap Shot'},
 'about': {'eventIdx': 346,
  'eventId': 867,
  'period': 3,
  'periodType': 'REGULAR',
  'ordinalNum': '3rd',
  'periodTime': '19:09',
  'periodTimeRemaining': '00:51',
  'dateTime': '2016-10-13T01:49:18Z',
  'goals': {'away': 4, 'home': 4}},
 'coordinates': {'x': 43.0, 'y': -18.0},
 'team': {'id': 9,
  'name': 'Ottawa Senators',
  'link': '/api/v1/teams/9',
  'triCode': 'OTT'}}

In [37]:
goal_exemple

{'players': [{'player': {'id': 8474068,
    'fullName': 'Kyle Turris',
    'link': '/api/v1/people/8474068'},
   'playerType': 'Scorer',
   'seasonTotal': 2},
  {'player': {'id': 8475913,
    'fullName': 'Mark Stone',
    'link': '/api/v1/people/8475913'},
   'playerType': 'Assist',
   'seasonTotal': 2},
  {'player': {'id': 8474578,
    'fullName': 'Erik Karlsson',
    'link': '/api/v1/people/8474578'},
   'playerType': 'Assist',
   'seasonTotal': 2},
  {'player': {'id': 8475883,
    'fullName': 'Frederik Andersen',
    'link': '/api/v1/people/8475883'},
   'playerType': 'Goalie'}],
 'result': {'event': 'Goal',
  'eventCode': 'OTT876',
  'eventTypeId': 'GOAL',
  'description': 'Kyle Turris (2) Snap Shot, assists: Mark Stone (2), Erik Karlsson (2)',
  'secondaryType': 'Snap Shot',
  'strength': {'code': 'EVEN', 'name': 'Even'},
  'gameWinningGoal': True,
  'emptyNet': False},
 'about': {'eventIdx': 354,
  'eventId': 876,
  'period': 4,
  'periodType': 'OVERTIME',
  'ordinalNum': 'OT',
 

# Data interpretation

In [16]:
season = load_processed_data(2016, samples=True)

Processing data... (1-2 minutes)
Season 2016 successfully loaded from file
Done!


In [11]:
# What are the strengh types ?

strengh_types = set()
for game in season.regulars:
  for play in game.plays:
    if play.result.strength:
      strengh_types.add(play.result.strength)

strengh_types

{'Even', 'Power Play', 'Short Handed'}

In [12]:
# What are the shot types ?

shot_types = set()
for game in season.regulars:
  for play in game.plays:
    res = play.result
    if (res.event == 'Goal' or res.event == 'Shot') and res.secondaryType:
      shot_types.add(res.secondaryType)

shot_types

{'Backhand',
 'Deflected',
 'Slap Shot',
 'Snap Shot',
 'Tip-In',
 'Wrap-around',
 'Wrist Shot'}

# To DataFrames

In [27]:
df = load_df_shots(2016)

In [28]:
df[df['Goal'] == True]

,Game Id,Periode,Temps écoulé,Equipe,Goal,X,Y,X-opp,Tireur,Gardien,Type,Filet Vide,Force,Net distance
6,1,1,08:21,TOR,True,-70.0,1.0,-89,Auston Matthews,Craig Anderson,Wrist Shot,False,Even,19.026298
11,1,1,10:26,OTT,True,82.0,3.0,89,Bobby Ryan,Frederik Andersen,Backhand,False,Even,7.615773
14,1,1,12:49,OTT,True,34.0,-1.0,89,Erik Karlsson,Frederik Andersen,Slap Shot,False,Even,55.009090
15,1,1,14:18,TOR,True,-76.0,-28.0,-89,Auston Matthews,Craig Anderson,Snap Shot,False,Even,30.870698
24,1,2,01:25,TOR,True,76.0,-14.0,89,Auston Matthews,Craig Anderson,Wrist Shot,False,Even,19.104973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74920,1230,2,04:10,EDM,True,-63.0,0.0,-89,Jordan Eberle,Richard Bachman,Wrist Shot,False,Even,26.000000
74927,1230,2,14:46,EDM,True,-81.0,-5.0,-89,Drake Caggiula,Richard Bachman,Wrist Shot,False,Even,9.433981
74935,1230,3,00:31,EDM,True,79.0,2.0,89,Jordan Eberle,Richard Bachman,Tip-In,False,Even,10.198039
74939,1230,3,01:50,EDM,True,73.0,2.0,89,Leon Draisaitl,Richard Bachman,Snap Shot,False,Even,16.124515


# Visualisation

## Number of shots, by type

In [29]:
ddf = df[df['Type'] != '']
df_g = ddf.groupby(['Type', 'Goal']).size().to_frame('Counts').reset_index()
df_g['Percentage'] = df_g['Counts'] / df_g.groupby('Type')['Counts'].transform('sum') * 100

px.bar(df_g,
       x='Type',
       y='Counts',
       color='Goal',
       title="Number of shots, by type",
       log_y=True,
       text=df_g['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x))
       ).update_xaxes(categoryorder='total descending')

## Distance to net

Nets are positioned at x = -89,89 and y = 0, the teams switch sides at each periods.

In [30]:
df_g = df.groupby(['Goal','Net distance']).size().to_frame('Counts').reset_index()
df_g['Bins'] = pd.cut(df_g['Net distance'], 12)
df_g = df_g.groupby(['Goal','Bins'], observed=True)['Counts'].sum().to_frame('Counts').reset_index()
df_g['Percentage'] = df_g['Counts'] / df_g.groupby('Bins', observed=True)['Counts'].transform('sum') * 100
df_g['Bins'] = df_g['Bins'].astype('str')

In [31]:
from plotly.graph_objs import XBins
px.bar(df_g,
       x='Bins',
       y='Counts',
       color='Goal',
       title="Number of shots, by distance",
       log_y=True,
       text=df_g['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x))
       )

Add boxplot

In [33]:
game = season.regulars[1]
df = game.to_df()
match_title = f'{game.home_team.name} VS {game.away_team.name}'

/Users/alexandreganito/Documents/Data science/Project/NHL/src/models/models.py:140: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



## Ring

In [34]:
import plotly.express as px
fig = px.scatter(df, x="x", y="y", animation_frame="event idx", range_x=[-100,100], range_y=[-42.5,42.5])

fig.update_traces(mode='markers',
                         marker_size=15,
                         marker_color="#111111")

fig.add_layout_image(
  source="https://raw.githubusercontent.com/udem-ift6758/project-template/main/figures/nhl_rink.png",
  xref="x",
  yref="y",
  x=-100,
  y=42.5,
  sizex=200,
  sizey=85,
  sizing="stretch",
  opacity=0.9,
  layer="below"
)
fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[-100, 100])
fig.update_yaxes(
    showline=False,
    zeroline=False,
    showgrid=False,
    range=[-42.5, 42.5],
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.update_layout(
  title={
        'text': match_title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
  autosize=False,
  template="plotly_white",)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["redraw"] = True

for id, fr in enumerate(fig.frames):
  fr.layout.title = df.loc[id]['description']

for step in fig.layout.sliders[0].steps:
    step["args"][1]["frame"]["redraw"] = True


fig.show()

In [ ]:
fig.write_html("plotly_demo_3.html", auto_play = False)

# Autres

In [ ]:
play = season.regulars[0].plays[51]

# Create figure
fig = go.Figure()

fig.add_trace(go.Scatter(x= [89], y=[0],
                         mode='markers',
                         marker_size=15,
                         marker_color="#111111"))

# Add images
fig.add_layout_image(
        dict(
            source="https://raw.githubusercontent.com/udem-ift6758/project-template/main/figures/nhl_rink.png",
            xref="x",
            yref="y",
            x=-100,
            y=42.5,
            sizex=200,
            sizey=85,
            sizing="stretch",
            opacity=0.9,
            layer="below")
)

fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[-100, 100])
fig.update_yaxes(
    showline=False,
    zeroline=False,
    showgrid=False,
    range=[-42.5, 42.5],
    scaleanchor = "x",
    scaleratio = 1,
  )

# Set templates
# fig.update_layout(template="plotly_white")
fig.update_layout(
  title={
        'text': play.result.description,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    template="plotly_white")

fig.show()

In [ ]:
from ipywidgets import interact, IntSlider
import plotly.graph_objects as go

plays = season.regulars[0].plays

positions = [Coordinates(x=1, y=2), Coordinates(x=3, y=4), Coordinates(x=5, y=6)]

def plot_coordinate(index: int):
    # Get the chosen coordinates

    play = plays[index]

    # play = game.allPlays[index]

    # coord = positions[index]

    # Create a scatter plot for the chosen coordinate
    fig = go.Figure(go.Scatter(x= [play.coordinates.x], y=[play.coordinates.y],
                         mode='markers',
                         marker_size=15,
                         marker_color="#111111"))

    # Set the range so it stays constant
    # fig.update_layout(
    #     xaxis=dict(range=[0, max([c.x for c in positions]) + 1]),
    #     yaxis=dict(range=[0, max([c.y for c in positions]) + 1])
    # )

    fig.add_layout_image(
            dict(
                source="https://dbdzm869oupei.cloudfront.net/img/vinylrugs/preview/32553.png",
                xref="x",
                yref="y",
                x=-100,
                y=42.5,
                sizex=200,
                sizey=85,
                sizing="stretch",
                opacity=0.9,
                layer="below")
    )

    fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[-100, 100])
    fig.update_yaxes(
        showline=False,
        zeroline=False,
        showgrid=False,
        range=[-42.5, 42.5],
        scaleanchor = "x",
        scaleratio = 1,
      )

    fig.update_layout(
      title={
            'text': play.description,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        autosize=False,
        template="plotly_white")

    fig.show()

# Use ipywidgets to select a specific index
interact(plot_coordinate, index=IntSlider(min=0, max=len(plays)-1, value=0))